In [20]:
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

In [21]:
phone_5_6 = pd.read_csv('/data-0/qibo/Gdata/oneMonth/phoneRecord/qb_phonerecord_1905_1906.csv')

/home/qibo/qb_vir_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
DIC = defaultdict(defaultdict)
for idx, gp in tqdm(phone_5_6.groupby('formatphone')):
    for idx2, pg2 in gp.groupby('create_date'):
        DIC[idx][idx2] = pg2.mobile.tolist()

  6%|▌         | 2072020/33533100 [1:18:13<18:29:04, 472.78it/s]

In [1]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')


In [7]:
dataset.num_features

1433

In [12]:
data = dataset.data

In [15]:
data.x.shape

torch.Size([2708, 1433])

In [18]:
data.edge_index

tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]])

In [8]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(20):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()